In [1]:
import asyncio
import logging
from pathlib import Path

from ai_agent.agent.service import GenericAgent
from ai_agent.agent.context import SimpleExecutionContext
from ai_agent.controller.service import Controller
from ai_agent.filesystem import FileSystem
from ai_agent.llm.factory import create_llm
from ai_agent.llm.views import LLMType
from ai_agent.registry import Registry
from ai_agent.registry_helper import register_all_actions

async def main():
    # Enable debug logging
    logging.basicConfig(level=logging.INFO)
    
    # Set up components
    llm = create_llm(LLMType.ANTHROPIC, model='claude-3-5-sonnet-20241022')
    file_system = FileSystem()
    
    # Create controller (creates its own registry internally)
    controller = Controller(context_param_types={'file_system': FileSystem})
    
    # Register actions with the controller's registry
    register_all_actions(controller.registry, file_system)
    
    # Create execution context
    execution_context = SimpleExecutionContext(
        file_system=file_system,
        llm=llm,
        registry=controller.registry,
    )
    
    # Create and run agent
    agent = GenericAgent(
        task="Create a hello.txt file with a friendly greeting",
        llm=llm,
        execution_context=execution_context,
        controller=controller,
        file_system=file_system,
    )
    
    history = await agent.run(max_steps=5)
    if history.last and history.last.model_output:
        print(f"Task completed: {history.last.model_output.is_done}")
        if history.last.model_output.is_done:
            # Check if hello.txt was created
            hello_file = Path("hello.txt")
            if hello_file.exists():
                print(f"✅ Success! hello.txt created: {hello_file.read_text()}")
            else:
                print("❌ Task marked done but hello.txt not found")
    else:
        print("Agent failed to complete the task")

await main()

INFO:ai_agent.GenericAgent:🌟 Agent initialized with ID: aea5a75b-43dd-4d40-a826-011cd978e6d8
INFO:ai_agent.GenericAgent:⏺ Starting AI agent execution
INFO:ai_agent.GenericAgent:  ⎿ Task: Create a hello.txt file with a friendly greeting
INFO:ai_agent.GenericAgent:  ⎿ Max steps: 5
INFO:ai_agent.GenericAgent:  ⎿ Agent settings: AgentSettings
INFO:ai_agent.GenericAgent:⏺ Step 1/5: Beginning agent reasoning cycle
INFO:ai_agent.GenericAgent:⏺ Preparing execution context for reasoning
INFO:ai_agent.GenericAgent:  ⎿ Context prepared successfully
INFO:ai_agent.GenericAgent:⏺ Querying LLM for next actions
INFO:ai_agent.GenericAgent:  ⎿ Sending 2 messages to LLM


📋 Available actions: 14 total


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:ai_agent.GenericAgent:  ⎿ LLM responded in 6.55s: 2 actions, done=True
INFO:ai_agent.GenericAgent:⏺ Agent reasoning process
INFO:ai_agent.GenericAgent:  ⎿ I need to create a simple text file with a friendly greeting message. This is a straightforward task that can be accomplished with a single write_file action.
INFO:ai_agent.GenericAgent:⏺ Planning next actions
INFO:ai_agent.GenericAgent:  ⎿ Goal: Create hello.txt file with friendly greeting
INFO:ai_agent.GenericAgent:⏺ LLM planned 2 actions
INFO:ai_agent.GenericAgent:  ⎿ Action 1: write_file
INFO:ai_agent.GenericAgent:  ⎿ Action 2: done
INFO:ai_agent.GenericAgent:  ⎿ LLM response received, parsing actions
INFO:ai_agent.GenericAgent:⏺ Executing planned actions
INFO:ai_agent.GenericAgent:  ⎿ Found 2 valid actions to execute
INFO:ai_agent.GenericAgent:⏺ Execute action 1/2: write_file
INFO:ai_agent.GenericAgent:  ⎿ Parameters: path=hello.txt, conte

Task completed: True
✅ Success! hello.txt created: Hello there! 👋

I hope you're having a wonderful day! Thanks for stopping by.

Best wishes!
